In [2]:
import os
import yaml

# Paths
ALL_TASKS_DIR = './all_tasks'       # Folder containing all your tasks
DESIRED_YAML_PATH = './desired.yaml'  # Shared desired.yaml at the same level

def load_yaml(file_path):
    """Load YAML safely."""
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

def create_label_index_mapping(original_data, desired_data):
    """Create a mapping from original label indices to desired label indices."""
    original_label_dict = {int(k): v for k, v in original_data['names'].items()}
    desired_label_dict = {int(k): v for k, v in desired_data['names'].items()}

    # Reverse dicts: name -> index
    desired_name_to_index = {v: k for k, v in desired_label_dict.items()}

    mapping = {}
    for orig_index, label_name in original_label_dict.items():
        if label_name in desired_name_to_index:
            mapping[orig_index] = desired_name_to_index[label_name]
        else:
            print(f"Label '{label_name}' (index {orig_index}) not in desired labels. Excluding.")
    return mapping

def remap_labels(file_path, label_index_mapping):
    """Apply the mapping to each line of the .txt file."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    remapped_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            print(f"[WARNING] Invalid line in file '{file_path}': {line.strip()}")
            continue
        
        try:
            label_index = int(parts[0])
        except ValueError:
            print(f"[WARNING] Could not parse label index in '{file_path}': {line.strip()}")
            continue
        
        if label_index in label_index_mapping:
            new_label_index = label_index_mapping[label_index]
            parts[0] = str(new_label_index)
            remapped_line = " ".join(parts)
            remapped_lines.append(remapped_line)
        else:
            # Label index not found in mapping → exclude
            print(f"[INFO] Label index {label_index} not in mapping for '{file_path}'. Excluding.")
    
    # Overwrite file with remapped lines
    with open(file_path, 'w') as file:
        file.write("\n".join(remapped_lines))

def process_task_folder(task_folder):
    """
    1. Load the folder's data_converted.yaml
    2. Load the shared desired.yaml
    3. Create index mapping
    4. Remap .txt files in 'labels' subfolder
    """
    data_converted_path = os.path.join(task_folder, 'data_converted.yaml')
    labels_dir = os.path.join(task_folder, 'labels')

    # Check that necessary files/folders exist
    if not os.path.isfile(data_converted_path):
        print(f"[WARNING] No data_converted.yaml in {task_folder}, skipping.")
        return
    if not os.path.isdir(labels_dir):
        print(f"[WARNING] No 'labels' directory in {task_folder}, skipping.")
        return

    # Load YAMLs
    original_data = load_yaml(data_converted_path)
    desired_data = load_yaml(DESIRED_YAML_PATH)

    # Build mapping
    label_mapping = create_label_index_mapping(original_data, desired_data)

    # Remap all .txt files
    annotation_files = [
        f for f in os.listdir(labels_dir) if f.endswith('.txt')
    ]
    for ann_file in annotation_files:
        file_path = os.path.join(labels_dir, ann_file)
        remap_labels(file_path, label_mapping)
    
    print(f"[INFO] Remapping completed for '{task_folder}'")

def main():
    """
    Iterate over each subdirectory in ALL_TASKS_DIR (each being a task),
    then remap labels using the above logic.
    """
    if not os.path.isdir(ALL_TASKS_DIR):
        print(f"[ERROR] Directory '{ALL_TASKS_DIR}' does not exist.")
        return

    subfolders = [
        os.path.join(ALL_TASKS_DIR, item) 
        for item in os.listdir(ALL_TASKS_DIR) 
        if os.path.isdir(os.path.join(ALL_TASKS_DIR, item))
    ]

    # Process each task folder
    for task_folder in subfolders:
        process_task_folder(task_folder)

if __name__ == "__main__":
    main()


Label 'RLY' (index 12) not in desired labels. Excluding.
Label 'RMY' (index 13) not in desired labels. Excluding.
Label 'RLR' (index 15) not in desired labels. Excluding.
Label 'RMR' (index 16) not in desired labels. Excluding.
Label 'SR' (index 22) not in desired labels. Excluding.
Label 'PMPL' (index 35) not in desired labels. Excluding.
Label 'GRLG' (index 51) not in desired labels. Excluding.
[INFO] Remapping completed for './all_tasks/task_ayan done_0_annotations_2025_01_13_10_36_27_coco 1.0'
[INFO] Remapping completed for './all_tasks/task_new annotation_laman_done_annotations_2025_01_13_08_27_43_coco 1.0'
Label 'RLR' (index 13) not in desired labels. Excluding.
Label 'RMR' (index 14) not in desired labels. Excluding.
[INFO] Remapping completed for './all_tasks/task_laman_done_2_annotations_2025_01_02_08_16_58_coco 1.0'
[INFO] Remapping completed for './all_tasks/task_ayan done_5_annotations_2025_01_02_11_37_59_coco 1.0'
Label 'GRLG' (index 64) not in desired labels. Excluding.
[